In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.python.keras as krs
from keras.layers import LSTM, Dropout, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler


2022-10-15 20:27:39.433712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 20:27:39.584981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 20:27:39.585009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-15 20:27:39.626457: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 20:27:40.448057: W tensorflow/stream_executor/platform/de

In [2]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [3]:
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [4]:
_data_len = len(_df.index)
_train_set_ratio, _test_set_ratio, _validation_set_ratio = .8, .1, .1
scaler = MinMaxScaler(feature_range=(0,1))

_train_set_len = int(_data_len * _train_set_ratio)
_test_set_len = int(_data_len * _test_set_ratio)
_validation_set_len = int(_data_len * _validation_set_ratio)

_train_set_end = _train_set_len
_test_set_end = _train_set_end + _test_set_len
_validation_set_end = _test_set_end + _validation_set_len


colnames = _df.columns.to_numpy()

#_data = _df.to_numpy().transpose()[1, :]
_data = _df.to_numpy().transpose()[1:5, :]
#_data = scaler.fit_transform(_data)


#train_set, test_set, validation_set = _df.iloc[:_train_set_end, :], _df.iloc[_train_set_end:_test_set_end, :], _df.iloc[_test_set_end:_validation_set_end, :]
train_set, test_set, validation_set = _data[:, :_train_set_end], _data[:, _train_set_end:_test_set_end], _data[:, _test_set_end:_validation_set_end]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
colnames

In [ ]:
train_set[:, :10]

In [ ]:
test_set[:, :10]

In [ ]:
validation_set[:, :10]

In [ ]:
train_set.shape

In [ ]:
def batches_create(data_array: np.array):
    _x_train = np.array([data_array[:, i:i+60].astype("float32") for i in range(data_array.shape[1]-60-1)])
    _y_train = np.array([data_array[:, i+60].astype("float32") for i in range(data_array.shape[1]-60-1)])
    print(_x_train.shape)
    return np.reshape(_x_train, (_x_train.shape[0], _x_train.shape[1], 60)), _y_train


In [ ]:
X_train, y_train = batches_create(train_set)
X_test, y_test = batches_create(test_set)
X_validation, y_validation = batches_create(validation_set)


In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
prediction_test = model.predict(np.array(X_validation))

In [ ]:
np.unique(prediction_test)

In [ ]:
#scaler.inverse_transform(prediction_test)

In [ ]:
y_test[0]

In [ ]:
np.array([X_test[0]])

In [ ]:
X_test.shape